In [ ]:
# try scale-free, double check on fat-tail
# final state -- degree distribution, # of components, 

# interesting outputs -- final degree distribution
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy.sparse as ss

from rewire import *


In [ ]:
N = 1000  # number of nodes 
graph_density = 5.0/N
mx_init = init_graph(N, graph_density)
print 'meanK=', mx_init.sum(axis=1).mean()
is_infected_init = (np.random.random(N) < .05)
print mx_init.sum()

In [ ]:
mx_init_sp = ss.csr_matrix(mx_init)


In [ ]:
def bf(self_state, neighs_infected, neighs_noninfected, global_infected):
    #return self_state * (neighs_infected + neighs_noninfected)
    #if not self_state:
    #    return np.inf
    return self_state * neighs_noninfected
    #return self_state * global_infected

#rdict = run_rewire(mx_init, is_infected_init, benefit_function=bf, 
#                   opts=dict(NUM_ITERS=5000, do_rewire=False, do_add=True, p_rewire=.1, p_transmit=.00005,
#                            iterate_only_infected=True, beta=5))
rdict = run_rewire(mx_init_sp, is_infected_init, benefit_function=bf, 
                   opts=dict(NUM_ITERS=10000, do_rewire=False, do_add=True, p_rewire=1, p_transmit=.0001,
                            iterate_only_infected=True, beta=10))

#rdict = run_rewire(mx_init, is_infected_init, benefit_function=bf, 
#                   opts=dict(NUM_ITERS=10000, do_rewire=False, do_add=True, p_rewire=1.0, p_transmit=.00002,
#                            iterate_only_infected=True, beta=100))

In [ ]:
def evangelical(self_state, neighs_infected, neighs_noninfected, global_infected):
    # missionary - simplest
    return self_state*global_infected
    # in addition to i-i to i-n switching, also switching from n-n to n-i -- why?
    # wannabe proszelysers?
rdict2 = run_rewire(mx_init_sp, is_infected_init, benefit_function=evangelical, 
                   opts=dict(NUM_ITERS=3000, do_rewire=True, do_add=False, p_rewire=1, p_transmit=.0005,
                            iterate_only_infected=True, beta=10))


In [ ]:
%run initplot.py

#with plt.style.context(['fivethirtyeight','seaborn-whitegrid']):
#with plt.rc_context({'lines.linewidth': 2}):
plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
ni=np.array(rdict['num_infected'],dtype='float')/N
plt.plot(ni, color='k')
plt.ylabel('Portion infected')
plt.title('Adding S-I links process')
plt.subplot(2,2,3)
plt.semilogy(1-ni, color='k')
plt.ylabel('Portion not infected')
#from matplotlib.ticker import MaxNLocator
#plt.gca().xaxis.set_major_locator( MaxNLocator(nbins = 3) )
plt.xlabel('Iteration')
plt.subplot(2,2,2)
ni=np.array(rdict2['num_infected'],dtype='float')/N
plt.plot(ni, color='k')
#plt.ylabel('Portion infected')
plt.title('Evangelical rewiring process')
plt.subplot(2,2,4)
plt.semilogy(1-ni, color='k')
#plt.ylabel('Portion not infected')
#plt.gca().xaxis.set_major_locator( MaxNLocator(nbins = 3) )
plt.xlabel('Iteration')
#plt.savefig('out/weird-superexponential.pdf', facecolor='white', bbox_inches='tight')

In [ ]:
plt.figure()
s=200
idot=np.array((ni[s:]-ni[:-s])/ni[s:])
plt.plot(idot)
plt.figure()
plt.plot(rdict2['mean_degree'])
plt.hold('on')
plt.plot(rdict2['mean_degree_inf'])
plt.legend(['<k>','<k_inf>'])
plt.ylim([0, plt.ylim()[1]])

## print rdict['mx'].sum()
print mx_init.sum()

In [ ]:
rws = np.array(rdict['rewire_counts_list'])
rwsndx, rwtypes = zip(*[(i, rt) for i, rt in enumerate(rdict['rewire_types']) if rt[-4:]!='none'])
#rwsndx, rwtypes = zip(*enumerate(rdict['rewire_types']))
plt.plot(rws[:,list(rwsndx)])
plt.legend(rwtypes, bbox_to_anchor=(.15,1))
